<font color='purple'>Привет, Артем! Спасибо за замечания и рекомендации. Постаралась все исправить.</font>

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Общее-впечатление" data-toc-modified-id="Общее-впечатление-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span><font color="orange">Общее впечатление</font></a></span></li><li><span><a href="#Общее-впечатление-(ревью-2)" data-toc-modified-id="Общее-впечатление-(ревью-2)-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span><font color="orange">Общее впечатление (ревью 2)</font></a></span></li><li><span><a href="#Общее-впечатление-(ревью-3)" data-toc-modified-id="Общее-впечатление-(ревью-3)-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span><font color="orange">Общее впечатление (ревью 3)</font></a></span></li></ul></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score, roc_curve
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from tqdm import notebook
import warnings
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
toxic_comments = pd.read_csv('/datasets/toxic_comments.csv')
toxic_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [3]:
toxic_comments.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
#Посмотрим сколько у нас токсичных/нектоксичных текстов
display(toxic_comments['toxic'].value_counts())
#Выведем соотношение
class_ratio = toxic_comments['toxic'].value_counts()[0] / toxic_comments['toxic'].value_counts()[1]
class_ratio

0    143106
1     16186
Name: toxic, dtype: int64

8.841344371679229

Вывод

Видим что классы несбалансированы, применем несколько способов балансировки и сравним их



In [5]:
#удаляю столбец Unnamed:
toxic_comments = toxic_comments.drop(['Unnamed: 0'], axis=1)

In [6]:
#ввожу функцию очищения текстов постов:
def clear_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)   
    text = ' '.join(text.split())
    return text

In [7]:
%%time
#очищаю тексты постов:
toxic_comments['text'] = toxic_comments['text'].apply(clear_text) 

CPU times: user 3.55 s, sys: 35.1 ms, total: 3.58 s
Wall time: 3.58 s


In [8]:
#ввожу функцию РОS-тэгирования слов:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,               #прилагательное
                "N": wordnet.NOUN,              #существительное
                "V": wordnet.VERB,              #глагол
                "R": wordnet.ADV                #наречие
               }  
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

#ввожу функцию леммализации тектов постов:
def lemm_text(text):
    text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]
    return ' '.join(text)

In [9]:
%%time
#леммализирую тексты постов:
toxic_comments['text'] = toxic_comments['text'].apply(lemm_text) 

CPU times: user 14min 51s, sys: 1min 26s, total: 16min 18s
Wall time: 16min 19s


In [10]:
from spacy.lang.en.stop_words import STOP_WORDS

type(STOP_WORDS)

set

In [11]:
# Разобьем выборку по отношению 50/20/20. Уменьшим количество кроссвалидаций до 3 из-за размера выборки.
target = toxic_comments['toxic']
features = toxic_comments.drop(['toxic'], axis=1)

features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                              target, 
                                                                              test_size=0.5, 
                                                                              random_state=1515)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, 
                                                                            target_valid, 
                                                                            test_size=0.5,
                                                                            random_state=1515)

#nltk.download('stopwords')
#stopwords = set(nltk_stopwords.words('english'))

#count_tf_idf = TfidfVectorizer(stop_words=list(STOP_WORDS))

#features_train = count_tf_idf.fit_transform(features_train['lemm_text'].values.astype('U'))
#features_valid = count_tf_idf.transform(features_valid['lemm_text'].values.astype('U'))
#features_test = count_tf_idf.transform(features_test['lemm_text'].values.astype('U'))
print(features_train.shape)
print(features_valid.shape)
print(features_test.shape)
cv_counts = 2 #ввожу функцию очищения текстов постов:
def clear_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)   
    text = ' '.join(text.split())
    return text

(79646, 1)
(39823, 1)
(39823, 1)


In [12]:
#смотрю соотношение 1/0 в выборках на примере target_train:
indices_1 = [i for i,x in enumerate(target_train) if x == 1]
count_1 = len(indices_1)

indices_0 = [i for i,x in enumerate(target_train) if x == 0]
count_0 = len(indices_0)

print('Доля значений 1 в тренировочной выборке:', len(indices_1) / (len(indices_1) + len(indices_0)))

Доля значений 1 в тренировочной выборке: 0.10129824473294327


Ресемплинг с уменшением класса 0

In [13]:
%%time
#уменьшаю кол-во 0 в выборках train:

comments_train = toxic_comments.iloc[target_train.index]
target_train_0 = comments_train[comments_train['toxic'] == 0]['toxic']
target_train_1 = comments_train[comments_train['toxic'] == 1]['toxic']


target_train_0_resample = target_train_0.sample(target_train_1.shape[0], random_state=12345)
target_train_resample = pd.concat([target_train_0_resample, target_train_1])

features_train_resample = toxic_comments.iloc[target_train_resample.index]

features_train_resample, target_train_resample = shuffle(features_train_resample,
                                                         target_train_resample,
                                                         random_state=12345)

features_train_resample = features_train_resample.text 

print('Соотношение 1/0 в тренировочной выборке:')
print(target_train_resample.value_counts(normalize=True))
print()
print(features_train_resample.shape)
print(target_train_resample.shape)

Соотношение 1/0 в тренировочной выборке:
0    0.5
1    0.5
Name: toxic, dtype: float64

(16136,)
(16136,)
CPU times: user 26.4 ms, sys: 4.23 ms, total: 30.7 ms
Wall time: 29.8 ms


Вывод

В ходе подготовки данных мы получили признаки для обучения, получили обучающую, валидационную и тестовую выборку.

Очистила тексты комментариев от ненужных знаков, леммализировала, убрала стоп-слова

Сбалансировала данные в целевом признаке

## Обучение

3.1. Логистическая регрессия

In [14]:
features_train = features_train.text

In [15]:
%%time

#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english', sublinear_tf=True)), 
                     ("lr", LogisticRegression())])
    
parameters = {'lr__solver': ('liblinear', 'saga','newton-cg', 'lbfgs'),
              'lr__C': (.1, 1, 5, 10),
              'lr__random_state': ([12345]),
              'lr__max_iter': ([200]),
              'lr__class_weight': (['balanced'])} #ну я поставила, хотя он тут не нужен
                                                  #я ж сама сбалансировала их
                                                  #нужен на несбалансированных данных, но у меня не получается их подать в модель
gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train, target_train)

mts = gscv.cv_results_['mean_test_score']
lr_train_f1 = max(mts)

print('F1 логистической регрессии =', round(lr_train_f1,2))
print('при параметрах', gscv.best_params_)
print()

#валидация:
predictions_valid = gscv.predict(features_valid.text)
lr_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 логистической регрессии на валидации =', round(lr_valid_f1,2))
print()
#кросc-валидация
lrbs=gscv.best_score_
print('Качество модели на кросс-валидации', lrbs)


F1 логистической регрессии = 0.76
при параметрах {'lr__C': 5, 'lr__class_weight': 'balanced', 'lr__max_iter': 200, 'lr__random_state': 12345, 'lr__solver': 'lbfgs'}

F1 логистической регрессии на валидации = 0.75

Качество модели на кросс-валидации 0.755329790108115
CPU times: user 7min 37s, sys: 5min 29s, total: 13min 7s
Wall time: 13min 8s


3.2. Дерево решений

In [16]:
%%time

#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')), 
                     ("dtc", DecisionTreeClassifier())])
    
parameters = {'dtc__max_depth': ([x for x in range(1, 25)]),
              'dtc__random_state': ([12345]), 
              'dtc__class_weight': (['balanced'])}

gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train, target_train)

mts = gscv.cv_results_['mean_test_score']
dtc_train_f1 = max(mts)

print('F1 дерева решений =', round(dtc_train_f1,2))
print('при параметрах', gscv.best_params_)
print()

#валидация:
predictions_valid = gscv.predict(features_valid.text)
dtc_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 дерева решений на валидации =', round(dtc_valid_f1,2))
print()
#кросc-валидация
dtbs=gscv.best_score_
print('Качество модели на кросс-валидации', dtbs)



F1 дерева решений = 0.63
при параметрах {'dtc__class_weight': 'balanced', 'dtc__max_depth': 23, 'dtc__random_state': 12345}

F1 дерева решений на валидации = 0.63

Качество модели на кросс-валидации 0.6334738586784482
CPU times: user 6min 35s, sys: 765 ms, total: 6min 35s
Wall time: 6min 36s


3.3. CatBoostClassifier

In [17]:
%%time

#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')), 
                     ("cbc", CatBoostClassifier())])
    
parameters = {'cbc__verbose': ([False]),
              'cbc__iterations': ([200]),
              'cbc__class_weights':([(1, 1), (1, 11)])} 
gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train, target_train)

mts = gscv.cv_results_['mean_test_score']
cbc_train_f1 = max(mts)

print('F1 CatBoostClassifier =', round(cbc_train_f1,2))
print('при параметрах', gscv.best_params_)
print()

#валидация:
predictions_valid = gscv.predict(features_valid.text)
cbc_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 CatBoostClassifier на валидации =', round(cbc_valid_f1,2))
print()
#кросc-валидация
cbs=gscv.best_score_
print('Качество модели на кросс-валидации', cbs)


F1 CatBoostClassifier = 0.73
при параметрах {'cbc__class_weights': (1, 1), 'cbc__iterations': 200, 'cbc__verbose': False}

F1 CatBoostClassifier на валидации = 0.73

Качество модели на кросс-валидации 0.7283102965329712
CPU times: user 23min 56s, sys: 13 s, total: 24min 8s
Wall time: 24min 16s


3.4 BERT

Обучение модели BERT можно посмотреть по ссылке https://colab.research.google.com/drive/11ozHURPSOQlegktOhgULARLnlmSOeDYH?usp=sharing. Данная модель обеспечивает качество F1 - 0.69. Ее рекомендовать не будем.

## Выводы

In [18]:
index = ['LogisticRegression',
         'DecisionTreeClassifier',
         'CatBoostClassifier']

data = {'F1 на обучающей выборке': [lr_train_f1,
                                    dtc_train_f1,
                                    cbc_train_f1],
        
        'F1 на валидационной выборке': [lr_valid_f1,
                                        dtc_valid_f1,
                                        cbc_valid_f1],
        'Качество модели на кросс-валидации': [lrbs,
                                        dtbs,
                                        cbs],}
f1_data = pd.DataFrame(data=data, index=index)

f1_data.sort_values(by='Качество модели на кросс-валидации', ascending=False)

,F1 на обучающей выборке,F1 на валидационной выборке,Качество модели на кросс-валидации
LogisticRegression,0.755330,0.753827,0.755330
CatBoostClassifier,0.728310,0.727960,0.728310
DecisionTreeClassifier,0.633474,0.633470,0.633474


Согласно проведенным расчетам моделей, лучше всего себя показала LogisticRegression. Проведем ее тестирование на тестовой выборке. 

In [19]:
#тестирование:
#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english', sublinear_tf=True)), 
                     ("lr", LogisticRegression())])
    
parameters = {'lr__solver': ('liblinear', 'saga','newton-cg', 'lbfgs'),
              'lr__C': (.1, 1, 5, 10),
              'lr__random_state': ([12345]),
              'lr__max_iter': ([200]),
              'lr__class_weight': (['balanced'])} #ну я поставила, хотя он тут не нужен
                                                  #я ж сама сбалансировала их
                                                  #нужен на несбалансированных данных, но у меня не получается их подать в модель
gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train, target_train)

mts = gscv.cv_results_['mean_test_score']
lr_train_f1 = max(mts)
predictions_test = gscv.predict(features_test.text)
lr_test_f1 = f1_score(target_test, predictions_test)
print('финальный (тестовый) тестовый F1 логистической регрессии =', round(lr_test_f1,2))

финальный (тестовый) тестовый F1 логистической регрессии = 0.76


Тестовая выборка показала результат близкий с результатом на кросс-валидации. Можем рекомендовать данную модель.

Аутсайдером среди моделей стала DecisionTreeClassifier, так как дала наименьшее F1.Модель BERT дала 0,69

Наилучшей моделью стала LogisticRegression, которая на тестировании показала F1 = 0.755. Поскольку требовалось найти модель классификации комментариев на позитивные и негативные со значением метрики качества F1 >= 0.75, рекомендовать могу LogisticRegression. 

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны